In [0]:
{
  "name": "MediSure_Main_Job",
  "email_notifications": {
    "no_alert_for_skipped_runs": false
  },
  "webhook_notifications": {},
  "timeout_seconds": 0,
  "trigger": {
    "pause_status": "PAUSED",
    "periodic": {
      "interval": 2,
      "unit": "HOURS"
    }
  },
  "max_concurrent_runs": 1,
  "tasks": [
    {
      "task_key": "01_git_to_bronze_file_transfer",
      "run_if": "ALL_SUCCESS",
      "notebook_task": {
        "notebook_path": "/Repos/icon.montalbar@gmail.com/medisure_jen/notebook/0_ingestion_git_to_databricks",
        "source": "WORKSPACE"
      },
      "timeout_seconds": 0,
      "email_notifications": {},
      "webhook_notifications": {}
    },
    {
      "task_key": "Stream_to_bronze_ingestion",
      "depends_on": [
        {
          "task_key": "01_git_to_bronze_file_transfer"
        }
      ],
      "run_if": "ALL_SUCCESS",
      "notebook_task": {
        "notebook_path": "/Repos/icon.montalbar@gmail.com/medisure_jen/notebook/01_streaming_data_bronze",
        "source": "WORKSPACE"
      },
      "timeout_seconds": 0,
      "email_notifications": {},
      "webhook_notifications": {}
    },
    {
      "task_key": "bronze_batch_insert",
      "depends_on": [
        {
          "task_key": "01_git_to_bronze_file_transfer"
        }
      ],
      "run_if": "ALL_SUCCESS",
      "notebook_task": {
        "notebook_path": "/Repos/icon.montalbar@gmail.com/medisure_jen/notebook/01_bronze_ingestion_batch_processing",
        "source": "WORKSPACE"
      },
      "timeout_seconds": 0,
      "email_notifications": {},
      "webhook_notifications": {}
    },
    {
      "task_key": "DLT_Silver_Data_Quality",
      "depends_on": [
        {
          "task_key": "bronze_batch_insert"
        },
        {
          "task_key": "Stream_to_bronze_ingestion"
        }
      ],
      "run_if": "ALL_SUCCESS",
      "pipeline_task": {
        "pipeline_id": "5b654246-e4fa-45d0-81c3-f58d8c929d08",
        "full_refresh": false
      },
      "timeout_seconds": 0,
      "email_notifications": {},
      "webhook_notifications": {}
    },
    {
      "task_key": "Gold_Layer_Pipeline",
      "depends_on": [
        {
          "task_key": "DLT_Silver_Data_Quality"
        }
      ],
      "run_if": "ALL_SUCCESS",
      "pipeline_task": {
        "pipeline_id": "c7ec8af0-13a1-4d5c-bab8-7ecf612afed1"
      },
      "timeout_seconds": 0,
      "email_notifications": {},
      "webhook_notifications": {}
    },
    {
      "task_key": "Fraud_monitoring",
      "depends_on": [
        {
          "task_key": "Gold_Layer_Pipeline"
        }
      ],
      "run_if": "ALL_SUCCESS",
      "notebook_task": {
        "notebook_path": "/Repos/icon.montalbar@gmail.com/medisure_jen/notebook/fraud_monitoring",
        "source": "WORKSPACE"
      },
      "timeout_seconds": 0,
      "email_notifications": {},
      "webhook_notifications": {}
    }
  ],
  "queue": {
    "enabled": true
  },
  "environments": [
    {
      "environment_key": "dbt-default",
      "spec": {
        "client": "3",
        "dependencies": [
          "dbt-databricks>=1.0.0,<2.0.0"
        ]
      }
    }
  ],
  "performance_target": "STANDARD",
  "run_as": {
    "user_name": "icon.montalbar@gmail.com"
  }
}